In [1]:
import os
from datetime import datetime

import numpy as np
from scipy.spatial import distance
from PIL import ImageFont

from fpdf import FPDF


In [3]:
scale = 10
font_sizes = [8, 11, 15]
font_paths = {
    "Arial": "fonts/arial.ttf",
    "Comic Sans MS": "fonts/comic.ttf",
    "Courier New": "fonts/cour.ttf",
    "Malgun Gothic": "fonts/malgun.ttf",
    "Times New Roman": "fonts/times.ttf",
}
fonts = {
    font_name: {font_size: ImageFont.truetype(font_path, font_size) for font_size in font_sizes}
    for font_name, font_path
    in font_paths.items()
}


In [18]:
def _make_cell_with_box(pdf: FPDF, tl: tuple[float, float], br: tuple[float, float], font_name: str, text: str):
    """Make a new cell in pdf.

    Parameters
    ----------
    pdf : FPDF
        FPDF object
    tl : tuple[int, int]
        Coordinate of top-left corner (x, y)
    br : tuple[int, int]
        Coordinate of bottom-right corner (x, y)
    font : str
        Font of the cell.
    text : str
        Text of the cell.
    """
    pdf.set_xy(*tl)
    w, h = br[0] - tl[0], br[1] - tl[1]  # 단위: mm

    d = {}
    for point, font in fonts[font_name].items():
        bbox = font.getbbox(text)  # 단위: px
        rendered_w = (bbox[2] - bbox[0]) * 0.34  # px -> mm. 실제로는 0.352778이라는데 재 보니 0.34에 가깝다.
        rendered_h = (bbox[3] - bbox[1]) * 0.34
        if rendered_w > w * 1.05:
            continue
        d[point] = distance.euclidean([w, h], [rendered_w, rendered_h])
    if d:
        size = min(d, key=d.get)
    else:
        size = font_sizes[0]
    pdf.set_font(font_name, size=size)
    pdf.cell(w, h, text, align='L')


In [19]:
pdf = FPDF(unit='mm')
for font_name, font_path in font_paths.items():
    pdf.add_font(font_name, fname=font_path)
pdf.add_page(format='A4')

# results: results in ocr.ipynb
# each_fonts: each_fonts in ocr.ipynb
for (bbox, text, prob), font in zip(results, each_fonts):
    (tl, tr, br, bl) = bbox
    tl = (tl[0] / scale, tl[1] / scale)
    br = (br[0] / scale, br[1] / scale)
    _make_cell_with_box(pdf, tl, br, font_name=font, text=text)

pdf.output("Output.pdf")


meta NOT subset; don't know how to subset; dropped


14.17233560090703
